# Семинар 9 - Методы построения оптического потока по последовательности изображений

**Этот семинар содержит оцениваемое домашнее задание**

***

Источник - https://habr.com/ru/post/201406/

$\textbf{Task statement}$: Оптический поток (ОП) – изображение видимого движения, представляющее собой сдвиг каждой точки (пикселя) между двумя изображениями.

По сути, он представляет собой поле скоростей. Суть ОП в том, что для каждой точки изображения $I_{t_0} (\vec{r})$ находится такой вектор сдвига $\delta \vec{r}$, чтобы было соответсвие между исходной точкой и точкой на следущем фрейме $I_{t_1} (\vec{r} + \delta \vec{r})$. В качестве метрики соответвия берут близость интенсивности пикселей, беря во внимание маленькую разницу по времени между кадрами: $\delta{t} = t_{1} - t_{0}$. В более точных методах точку можно привязывать к объекту на основе, например, выделения ключевых точек, а также считать градиенты вокруг точки, лапласианы и проч.

$\textbf{For what}$: Определение собственной скорости, Определение локализации, Улучшение методов трекинга объектов, сегментации, Детектирование событий, Сжатие видеопотока и проч.

![](data/tennis.png)

Разделяют 2 вида оптического потока - плотный (dense) [Farneback method, neural nets], работающий с целым изображением, и выборочный (sparse) [Lucas-Kanade method], работающий с ключевыми точками

In [14]:
!wget https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4 -O data/slow_traffic_small.mp4

--2025-05-18 19:19:44--  https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4
Resolving www.bogotobogo.com (www.bogotobogo.com)... 173.254.30.214
Connecting to www.bogotobogo.com (www.bogotobogo.com)|173.254.30.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2018126 (1.9M) [video/mp4]
Saving to: ‘data/slow_traffic_small.mp4’

data/slow_traffic_s 100%[===================>]   1.92M  4.95MB/s    in 0.4s    

2025-05-18 19:19:45 (4.95 MB/s) - ‘data/slow_traffic_small.mp4’ saved [2018126/2018126]



In [2]:
import cv2
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import IPython

%matplotlib inline

## Lucas-Kanade (sparse)

Пусть $I_{1} = I(x, y, t_{1})$ интенсивность в некоторой точке (x, y) на первом изображении (т. е. в момент времени t). На втором изображении эта точка сдвинулась на (dx, dy), при этом прошло время dt, тогда $I_{2} = I(x + dx, y + dx, t_{1} + dt) \approx I_{1} + I_{x}dx + I_{y}dy +  I_{t}dt$. Из постановки задачи следует, что интенсивность пикселя не изменилась, тогда $I_{1} = I_{2}$. Далее определяем $dx, dy$.

Самое простое решение проблемы – алгоритм Лукаса-Канаде. У нас же на изображении объекты размером больше 1 пикселя, значит, скорее всего, в окрестности текущей точки у других точек будут примерно такие же сдвиги. Поэтому мы возьмем окно вокруг этой точки и минимизируем (по МНК) в нем суммарную погрешность с весовыми коэффициентами, распределенными по Гауссу, то есть так, чтобы наибольший вес имели пиксели, ближе всего находящиеся к исследуемому.

**Полезные материалы:**
- цикл видео-лекций от First Principles of Computer Vision, посвященный Optical Flow и алгоритму Lucas-Kanade: https://youtube.com/playlist?list=PL2zRqk16wsdoYzrWStffqBAoUY8XdvatV

### Вопрос 1

Перечислите три основных предположения, на которых базируется метод Lucas-Kanade. Почему каждое из них важно для корректной работы алгоритма?

**Ответ:**
1. Постоянство интенсивности пикселя:

Интенсивность (яркость) пикселя остается неизменной между двумя последовательными кадрами;

2. Малость смещения:

Сдвиги пикселей между кадрами малы (в пределах нескольких пикселей), т.е. можно использовать первый порядок разложения в ряд Тейлора;

3. Пространственная когерентность:

В маленькой окрестности предполагается, что все пиксели движутся одинаково, то есть имеют одинаковый вектор оптического потока.

### Вопрос 2

Объясните, зачем нужен пирамидальный подход в алгоритме Lucas-Kanade. Какую проблему он решает и как именно?

**Ответ:**

Пирамидальный подход нужен для обработки больших сдвигов между кадрами, которые нарушают предположение о малости движения.

Он решает проблему больших движений объектов, которые нельзя корректно аппроксимировать линейно на исходном разрешении.

Принцип работы:
1. Строятся пирамиды изображений, т.е. последовательность уменьшенных копий;
2. Оптический поток сначала оценивается на низком разрешении, где сдвиги визуально меньше;
3. Затем этот поток пропагируется и уточняется на более высоких уровнях до оригинального разрешения.

### Вопрос 3

С какими проблемами может столкнуться алгоритм Lucas-Kanade при отслеживании точек на видео? Назовите минимум три ограничения.

**Ответ:**

1. Быстрые и большие движения (нарушается предположение о малом смещении);
2. Однородные или текстурно-пустые области, т.к. в них невозможно надёжно вычислить градиенты (матрица системы становится вырожденной);
3. Изменения освещения или яркости (Нарушается предположение о постоянстве интенсивности).

### Задание 1

Напишите реализацию Лукаса-Канаде c помощью numpy и cv2. Сравните с реализацией `cv2.calcOpticalFlowPyrLK`.

In [3]:
def build_image_pyramid(image, num_levels, scale_factor=0.5):
    pyramid = [image.copy()]
    current_image = image.copy()

    for _ in range(1, num_levels):
        new_size = (int(current_image.shape[1] * scale_factor), int(current_image.shape[0] * scale_factor))
        current_image = cv2.resize(current_image, new_size, interpolation=cv2.INTER_LINEAR)
        pyramid.append(current_image)

    return pyramid

def compute_image_gradients(image):
    Ix = cv2.Sobel(image, cv2.CV_64F, dx=1, dy=0, ksize=3)
    Iy = cv2.Sobel(image, cv2.CV_64F, dx=0, dy=1, ksize=3)
    return Ix, Iy


In [4]:
def compute_lk_optical_flow_point(Ix, Iy, It, window_size=5):
    A = np.array([
        [np.sum(Ix * Ix), np.sum(Ix * Iy)],
        [np.sum(Ix * Iy), np.sum(Iy * Iy)]
    ])

    b = np.array([
        -np.sum(Ix * It),
        -np.sum(Iy * It)
    ])

    # Проверяем обусловленность матрицы A через собственные значения
    try:
        eigenvalues = np.linalg.eigvals(A)
        if np.min(eigenvalues) < 1e-4 or np.isnan(eigenvalues).any():
            return None, None

        # Решаем систему уравнений
        flow = np.linalg.solve(A, b)
        return flow[0], flow[1]
    except np.linalg.LinAlgError:
        return None, None

In [5]:
def compute_lk_optical_flow_for_patch(prev_patch, curr_patch, window_size=5):
    # Пространственные градиенты
    Ix, Iy = compute_image_gradients(prev_patch)
    # 2. Временной градиент
    It = curr_patch - prev_patch
    # 3. ОП
    u, v = compute_lk_optical_flow_point(Ix, Iy, It, window_size=window_size)
    return u, v

In [7]:
def track_point_with_pyramid_lk(prev_pyramid, curr_pyramid, point, window_size=15, max_iterations=10, epsilon=0.01):

    num_levels = len(prev_pyramid)
    point = np.array([point[0], point[1]], dtype=np.float32)

    # Начальное смещение
    flow = np.zeros(2, dtype=np.float32)

    # Обрабатываем уровни пирамиды от верхнего (маленького) к нижнему (большому)
    for level in range(num_levels - 1, -1, -1):
        # Масштабируем точку для текущего уровня
        scale = 1.0 / (2 ** level) if level > 0 else 1.0
        scaled_point = point * scale

        # Текущие координаты точки с учетом уже найденного потока
        current_point = scaled_point + flow * scale

        # Получаем изображения текущего уровня
        prev_img = prev_pyramid[level]
        curr_img = curr_pyramid[level]

        # Итеративно уточняем позицию
        for _ in range(max_iterations):
            # Округляем координаты для извлечения патча
            x, y = int(round(current_point[0])), int(round(current_point[1]))

            # Проверяем, что точка находится внутри изображения с учетом окна
            half_window = window_size // 2
            if (y - half_window < 0 or y + half_window >= prev_img.shape[0] or
                x - half_window < 0 or x + half_window >= prev_img.shape[1]):
                return None

            # Извлекаем патчи из предыдущего и текущего кадров
            prev_patch = prev_img[y-half_window:y+half_window+1, x-half_window:x+half_window+1]
            curr_patch = curr_img[y-half_window:y+half_window+1, x-half_window:x+half_window+1]

            # Проверяем, что патчи имеют правильный размер
            if prev_patch.shape[0] != window_size or prev_patch.shape[1] != window_size:
                return None

            # Вычисляем оптический поток для патча
            delta_flow = compute_lk_optical_flow_for_patch(prev_patch, curr_patch, window_size)

            # Если не удалось вычислить поток, прекращаем отслеживание
            if delta_flow[0] is None:
                return None

            # Обновляем позицию
            current_point += np.array(delta_flow)

            # Проверяем условие сходимости
            if abs(delta_flow[0]) < epsilon and abs(delta_flow[1]) < epsilon:
                break

        # Обновляем общий поток для следующего уровня
        flow = (current_point - scaled_point)

    # Возвращаем итоговую точку
    return (point[0] + flow[0], point[1] + flow[1])

In [8]:
def lucas_kanade_optical_flow(prev_frame, curr_frame, points,
                             window_size=15, num_pyramid_levels=3,
                             max_iterations=100, epsilon=0.1):
    # Преобразуем входные кадры в полутоновые, если они цветные
    if len(prev_frame.shape) == 3:
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    else:
        prev_gray = prev_frame

    if len(curr_frame.shape) == 3:
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    else:
        curr_gray = curr_frame

    # Нормализуем изображения к диапазону [0, 1]
    prev_gray = prev_gray.astype(np.float32) / 255.0
    curr_gray = curr_gray.astype(np.float32) / 255.0

    # Создаем пирамиды изображений
    prev_pyramid = build_image_pyramid(prev_gray, num_pyramid_levels)
    curr_pyramid = build_image_pyramid(curr_gray, num_pyramid_levels)

    # Подготавливаем массивы для результатов
    new_points = np.zeros_like(points, dtype=np.float32)
    status = np.zeros(len(points), dtype=np.int32)

    # Обрабатываем каждую точку
    for i, point in enumerate(points):
        # Отслеживаем точку с помощью пирамидального LK
        new_point = track_point_with_pyramid_lk(
            prev_pyramid, curr_pyramid, point,
            window_size, max_iterations, epsilon
        )

        # Сохраняем результат и статус
        if new_point is not None:
            new_points[i] = new_point
            status[i] = 1  # Успешное отслеживание
        else:
            new_points[i] = point  # Сохраняем исходную точку
            status[i] = 0  # Неуспешное отслеживание

    return new_points, status

In [9]:
def demo_optical_flow(video_path='data/slow_traffic_small.mp4', output_path='output_my_LK.mp4'):
    """
    Демонстрация работы алгоритма на видео.

    Args:
        video_path: Путь к входному видео
        output_path: Путь для сохранения результата
    """
    # Открываем видео
    cap = cv2.VideoCapture(video_path)

    # Получаем параметры видео
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Настраиваем запись выходного видео
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Параметры для обнаружения углов Shi-Tomasi
    feature_params = dict(
        maxCorners=100,
        qualityLevel=0.3,
        minDistance=7,
        blockSize=7
    )

    # Берем первый кадр и находим в нем углы
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
    p0 = p0.reshape(-1, 2)  # Преобразуем в формат [[x1, y1], [x2, y2], ...]

    # Сохраняем изначальные точки для отслеживания через все видео
    initial_points = p0.copy()

    # Создаем маску для рисования
    mask = np.zeros_like(old_frame)

    # Создаем случайные цвета для визуализации
    color = np.random.randint(0, 255, (len(p0), 3))

    from tqdm import tqdm
    for i in tqdm(range(length - 1)):  # -1 потому что первый кадр мы уже прочитали
        ret, frame = cap.read()

        if not ret:
            print('No frames grabbed!')
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Вычисляем оптический поток с помощью нашей реализации
        p1, st = lucas_kanade_optical_flow(
            old_gray,
            frame_gray,
            p0,
            window_size=15,
            num_pyramid_levels=3
        )

        # Выбираем хорошие точки
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Рисуем треки
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new
            c, d = old
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i % len(color)].tolist(), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, color[i % len(color)].tolist(), -1)

        # Объединяем кадр и маску
        img = cv2.add(frame, mask)

        # Записываем результат
        out.write(img)

        # Обновляем предыдущий кадр
        old_gray = frame_gray.copy()

        # Обновляем точки, но только те, которые успешно отслежены
        p0[st == 1] = good_new

    # Освобождаем ресурсы
    cap.release()
    out.release()

    print(f"Результат сохранен в {output_path}")
    return output_path

In [15]:
result_path = demo_optical_flow(video_path='data/slow_traffic_small.mp4', output_path='output_my_LK.mp4')

100%|██████████| 913/913 [00:20<00:00, 44.31it/s]

Результат сохранен в output_my_LK.mp4


### Релизация OpenCV - cv2.calcOpticalFlowPyrLK

In [16]:
def demo_optical_flow_opencv(video_path='data/slow_traffic_small.mp4', output_path='output_my_LK.mp4'):
    """
    Демонстрация работы алгоритма на видео с использованием cv2.calcOpticalFlowPyrLK.

    Args:
        video_path: Путь к входному видео
        output_path: Путь для сохранения результата
    """
    # Открываем видео
    cap = cv2.VideoCapture(video_path)

    # Получаем параметры видео
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Настраиваем запись выходного видео
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Параметры для обнаружения углов Shi-Tomasi
    feature_params = dict(
        maxCorners=100,
        qualityLevel=0.3,
        minDistance=7,
        blockSize=7
    )

    # Параметры для Lucas-Kanade оптического потока
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=3,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
    )

    # Берем первый кадр и находим в нем углы
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

    # Создаем маску для рисования
    mask = np.zeros_like(old_frame)

    # Создаем случайные цвета для визуализации
    color = np.random.randint(0, 255, (100, 3))

    from tqdm import tqdm
    for i in tqdm(range(length - 1)):  # -1 потому что первый кадр мы уже прочитали
        ret, frame = cap.read()

        if not ret:
            print('No frames grabbed!')
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Вычисляем оптический поток с помощью встроенной функции cv2.calcOpticalFlowPyrLK
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Выбираем хорошие точки
        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

        # Рисуем треки
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i % len(color)].tolist(), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, color[i % len(color)].tolist(), -1)

        # Объединяем кадр и маску
        img = cv2.add(frame, mask)

        # Записываем результат
        out.write(img)

        # Обновляем предыдущий кадр
        old_gray = frame_gray.copy()

        # Обновляем точки, но только те, которые успешно отслежены
        p0 = good_new.reshape(-1, 1, 2)

    # Освобождаем ресурсы
    cap.release()
    out.release()

    print(f"Результат сохранен в {output_path}")
    return output_path

In [17]:
result_path = demo_optical_flow_opencv(video_path='data/slow_traffic_small.mp4', output_path='output_opencv_LK.mp4')

100%|██████████| 913/913 [00:05<00:00, 163.47it/s]

Результат сохранен в output_opencv_LK.mp4


### Задание 2

В базовой реализации у кода есть одна важная проблема - ключевые точки инициализируются единожды. В реальных задачах необходимо отслеживать точки, которые исчезают из кадра и появляются в других местах. Реализуйте механизм, который будет отслеживать точки, которые пропадают из кадра и добавлять новые точки в те места, где они появляются. Для этого вам нужно будет реализовать механизм поиска новых точек на изображении.

In [19]:
def demo_optical_flow_enhanced(video_path='data/slow_traffic_small.mp4', output_path='output_my_LK_enhanced.mp4'):
    """
    Демонстрация работы алгоритма на видео.

    Args:
        video_path: Путь к входному видео
        output_path: Путь для сохранения результата
    """
    # Открываем видео
    cap = cv2.VideoCapture(video_path)

    # Получаем параметры видео
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Настраиваем запись выходного видео
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Параметры для обнаружения углов Shi-Tomasi
    feature_params = dict(
        maxCorners=100,
        qualityLevel=0.3,
        minDistance=7,
        blockSize=7
    )

    # Берем первый кадр и находим в нем углы
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
    p0 = p0.reshape(-1, 2)  # Преобразуем в формат [[x1, y1], [x2, y2], ...]

    # Создаем маску для рисования треков
    mask = np.zeros_like(old_frame)

    # Создаем случайные цвета для визуализации
    color = np.random.randint(0, 255, (500, 3))  # Увеличиваем размер массива цветов

    # Минимальное число точек, при котором необходимо искать новые
    min_points = 20

    # Для хранения всех идентификаторов точек
    point_ids = np.arange(len(p0))
    next_id = len(p0)

    from tqdm import tqdm
    for i in tqdm(range(length - 1)):  # -1 потому что первый кадр мы уже прочитали
        ret, frame = cap.read()

        if not ret:
            print('No frames grabbed!')
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Вычисляем оптический поток с помощью нашей реализации
        p1, st = lucas_kanade_optical_flow(
            old_gray,
            frame_gray,
            p0,
            window_size=15,
            num_pyramid_levels=3
        )

        # Выбираем хорошие точки
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        good_ids = point_ids[st == 1]

        # Обновляем массивы точек и их идентификаторов
        p0 = good_new
        point_ids = good_ids

        # Проверяем, нужно ли добавить новые точки
        if len(p0) < min_points:
            # Создаем маску, исключающую области вокруг существующих точек
            mask_points = np.zeros_like(old_gray)

            for x, y in p0:
                cv2.circle(mask_points, (int(x), int(y)), 10, 255, -1)  # Радиус 10 пикселей

            mask_points = cv2.bitwise_not(mask_points)

            # Ищем новые точки, исключая области с существующими точками
            new_features = cv2.goodFeaturesToTrack(
                frame_gray,
                mask=mask_points,
                maxCorners=100 - len(p0),  # Дополняем до максимального количества
                qualityLevel=0.2,          # Немного снижаем требования к качеству
                minDistance=7,
                blockSize=7
            )

            if new_features is not None:
                new_features = new_features.reshape(-1, 2)

                # Создаем новые идентификаторы для новых точек
                new_ids = np.arange(next_id, next_id + len(new_features))
                next_id += len(new_features)

                # Добавляем новые точки и их идентификаторы
                p0 = np.vstack([p0, new_features]) if len(p0) > 0 else new_features
                point_ids = np.concatenate([point_ids, new_ids]) if len(point_ids) > 0 else new_ids

                # Обновляем массив цветов, если необходимо
                if next_id > len(color):
                    new_colors = np.random.randint(0, 255, (500, 3))
                    color = np.vstack([color, new_colors])

        # Рисуем треки
        for i, (new, old, point_id) in enumerate(zip(good_new, good_old, good_ids)):
            a, b = new
            c, d = old
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[point_id % len(color)].tolist(), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, color[point_id % len(color)].tolist(), -1)

        # Объединяем кадр и маску
        img = cv2.add(frame, mask)

        # Записываем результат
        out.write(img)

        # Обновляем предыдущий кадр
        old_gray = frame_gray.copy()

    # Освобождаем ресурсы
    cap.release()
    out.release()

    print(f"Результат сохранен в {output_path}")
    return output_path

In [20]:
demo_optical_flow_enhanced()

100%|██████████| 913/913 [00:51<00:00, 17.80it/s]

Результат сохранен в output_my_LK_enhanced.mp4


'output_my_LK_enhanced.mp4'

### Вопрос 4

В чем основное отличие разреженного (sparse) оптического потока Lucas-Kanade от плотного (dense) оптического потока (например, метода Farneback)?

**Ответ:**

Разреженный поток (sparse) вычисляется только в отдельных ключевых точках, плотный поток (dense) вычисляется для каждого пикселя изображения.


## Farneback (dense)

Метод Farneback носит несколько более глобальный характер, чем метод Лукаса-Канаде. Он опирается на предположение о том, что на всем изображении оптический поток будет достаточно гладким.

# Вопрос 5

Перечислите основные шаги алгоритма Farneback для расчета оптического потока.

**Ответ:**

# Алгоритм Фарнебэка для расчета оптического потока

Алгоритм Farneback оценивает **плотный оптический поток** между двумя кадрами, моделируя локальные области с помощью квадратичных полиномов и итеративно уточняя смещения.

## 1. Полиномиальная аппроксимация

Каждое окно изображения $I(x)$, где $x = (x, y)$, аппроксимируется квадратичной функцией:

$$
I(x) \approx x^T A x + b^T x + c
$$

где:
- $A \in \mathbb{R}^{2 \times 2}$ — матрица кривизны;
- $b \in \mathbb{R}^2$ — вектор градиента;
- $c \in \mathbb{R}$ — скалярное смещение.

## 2. Предположение о смещении

Предполагается, что смещение $d = (u, v)$ между кадрами влияет на интенсивность во втором кадре:

$$
J(x) = I(x - d) \approx (x - d)^T A' (x - d) + b'^T (x - d) + c'
$$

## 3. Линеаризация уравнения

Разность аппроксимаций выражается через $d$:

$$
\Delta I(x) \approx (A + A') d + (b - b')
$$

## 4. Метод наименьших квадратов

Для минимизации ошибки решается система:

$$
d = (A^T W A)^{-1} A^T W b
$$

где $W$ — весовая матрица (например, гауссово окно), учитывающая вклад пикселей.

## 5. Весовая агрегация

Используется **Гауссов фильтр** для сглаживания оценок коэффициентов полинома. Это повышает устойчивость алгоритма к шуму и локальным выбросам.

## 6. Многомасштабный подход

Строится **пирамида изображений**:

- Поток сначала оценивается на самом грубом уровне (низкое разрешение);
- Затем он интерполируется и используется как инициализация на следующем уровне;
- Уточнение продолжается до самого высокого разрешения.

## 7. Итеративное уточнение

На каждом уровне пирамиды проводится несколько итераций уточнения потока $d$ для повышения точности.

---

## Вывод

Алгоритм Фарнебэка сочетает:
- Квадратичную полиномиальную аппроксимацию;
- Метод наименьших квадратов с весами;
- Гауссовое сглаживание;
- Многомасштабный анализ.

### Вопрос 6

Каким образом в методе Farneback обрабатываются большие смещения объектов между кадрами?

**Ответ:**

Для обработки больших смещений в методе Farneback используется **многомасштабный (пирамидальный) подход**:

- Строится **гауссова пирамида изображений**: на каждом следующем уровне изображение уменьшается (разрешение снижается);
- **Оценка оптического потока начинается с самого грубого уровня** (малого разрешения), где большие смещения превращаются в относительно маленькие;
- Затем поток **интерполируется** на следующий, более детализированный уровень и **используется как инициализация**;
- На каждом уровне проводится **итеративное уточнение** потока.

Таким образом, даже при значительных перемещениях объектов, метод способен постепенно приблизиться к корректной оценке смещений, избегая локальных минимумов.

In [21]:
def demo_optical_flow_farneback_opencv(video_path='data/slow_traffic_small.mp4', output_path='output_Farneback.mp4'):
    """
    Демонстрация работы алгоритма плотного оптического потока Farneback на видео.

    Args:
        video_path: Путь к входному видео
        output_path: Путь для сохранения результата
    """
    # Открываем видео
    cap = cv2.VideoCapture(video_path)

    # Получаем параметры видео
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Настраиваем запись выходного видео
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Берем первый кадр и преобразуем его в оттенки серого
    ret, frame1 = cap.read()
    if not ret:
        print('Не удалось прочитать видео')
        return None

    prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

    # Создаем HSV-изображение для визуализации потока
    hsv = np.zeros_like(frame1)
    hsv[..., 1] = 255  # Насыщенность устанавливаем на максимум

    from tqdm import tqdm
    for i in tqdm(range(length - 1)):  # -1 потому что первый кадр мы уже прочитали
        ret, frame2 = cap.read()

        if not ret:
            print('No frames grabbed!')
            break

        next_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Вычисляем оптический поток методом Farneback
        # Параметры:
        # - 0.5: коэффициент масштабирования для пирамиды изображений
        # - 3: кол-во уровней пирамиды
        # - 15: размер окна для усреднения
        # - 3: число итераций на каждом уровне пирамиды
        # - 5: размер окна для полиномиальной аппроксимации
        # - 1.2: стандартное отклонение для сглаживания
        flow = cv2.calcOpticalFlowFarneback(
            prvs, next_frame, None,
            0.5, 3, 15, 3, 5, 1.2, 0
        )

        # Преобразуем векторы потока из декартовых координат в полярные
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Кодируем направление потока как оттенок (hue)
        hsv[..., 0] = ang * 180 / np.pi / 2

        # Кодируем величину потока как яркость (value)
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

        # Преобразуем HSV в BGR для отображения
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        # Записываем результат
        out.write(bgr)

        # Обновляем предыдущий кадр
        prvs = next_frame

    # Освобождаем ресурсы
    cap.release()
    out.release()

    print(f"Результат сохранен в {output_path}")
    return output_path

In [22]:
result_path = demo_optical_flow_farneback_opencv(video_path='data/slow_traffic_small.mp4', output_path='output_opencv_farneback.mp4')

100%|██████████| 913/913 [01:31<00:00, 10.03it/s]

Результат сохранен в output_opencv_farneback.mp4


### Вопрос 7

Как влияет предварительная обработка изображений (фильтрация шума, выравнивание гистограмм) на качество оптического потока, получаемого методом Farneback? Предложите оптимальный пайплайн предобработки.

**Ответ:**

Предварительная обработка изображений повышает устойчивость и точность оптического потока, особенно при использовании метода Farneback, который чувствителен к шуму и локальным изменениям контраста.

#### Влияние предобработки:

- **Фильтрация шума** (например, Gaussian Blur):
  - Устраняет высокочастотный шум, который может искажать локальные полиномиальные аппроксимации;
  - Улучшает устойчивость оценки градиентов и матрицы $A$.

- **Выравнивание гистограмм**:
  - Нормализует контраст на изображении, особенно полезно при переменном освещении;
  - Повышает качество сопоставления между окнами в разных кадрах.

- **Градации серого (grayscale)**:
  - Метод Farneback работает с интенсивностями, поэтому перевод в оттенки серого упрощает обработку и снижает размер входных данных.

---

### Оптимальный пайплайн предобработки для Farneback:

1. **Приведение к градациям серого**:
   ```python
   gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

2. **Гауссово сглаживание**:
   ```python
   blurred = cv2.GaussianBlur(gray, (5, 5), sigmaX=1.5)

3. **Выравнивание гистограммы:**:
   ```python
   equalized = cv2.equalizeHist(blurred)\

4. **Передача в cv2.calcOpticalFlowFarneback()**:
   ```python
   flow = cv2.calcOpticalFlowFarneback(prev=equalized1, next=equalized2, ...)
